# Kubernetes

# Kubernetes Version (Current)

In [ ]:
from kubernetesTool import executeKubectlCmd
from kubernetesTool import toCmd
from kubernetesTool import executeLocalCmd
from kubernetesTool import createConfig
from kubernetesTool import getPort
from kubernetesTool import executePeerCmd

## check status peers

In [77]:
from kubernetesTool import executeKubectlCmd
_res = executeKubectlCmd(['get', 'pods', '-o', 'json'], decode='json')

for item in _res['items']:
    labels=item['metadata']['labels']
    #status=item['status']['initContainerStatuses'][0]
    #print(labels,status['state'],status['restartCount'],status['name'],status['ready'])
    print(labels,item['status']['phase'])
    #print(labels)

{'app': 'aabby-0', 'pod-template-hash': '6859c54c76'} Running
{'app': 'aabby-1', 'pod-template-hash': '5ff5b8677f'} Running
{'app': 'cc-deployer', 'pod-template-hash': '55c958b4d4'} Running
{'app': 'hl-fabric-data-share', 'pod-template-hash': '6849487f76'} Running
{'app': 'invoice-chaincode-external', 'pod-template-hash': '847cd44954'} Running
{'app': 'invoice-net-chaincode-external', 'pod-template-hash': '576f8cbdd'} Running
{'app': 'orderer-org1-scray-org', 'pod-template-hash': '5cb87587c4'} Running
{'app': 'peer80', 'pod-template-hash': '8774cd6cb'} Running
{'app': 'peer81', 'pod-template-hash': '66554b66f'} Running


In [103]:
kubernetespeer('invoice-chaincode-external').__dict__

Error from server (NotFound): services "invoice-chaincode-external" not found
Error from server (NotFound): services "invoice-chaincode-external" not found


{'name': 'invoice-chaincode-external',
 'pod_name': 'invoice-chaincode-external-847cd44954-w2ddx',
 'listen_port': 'Command \'[\'/home/jovyan/work/usr/bin/kubectl\', \'get\', \'service\', \'invoice-chaincode-external\', \'-o\', \'jsonpath="{.spec.ports[?(@.name==\\\'peer-listen\\\')].nodePort}"\']\' returned non-zero exit status 1.',
 'gossip_port': 'Command \'[\'/home/jovyan/work/usr/bin/kubectl\', \'get\', \'service\', \'invoice-chaincode-external\', \'-o\', \'jsonpath="{.spec.ports[?(@.name==\\\'peer-gossip\\\')].nodePort}"\']\' returned non-zero exit status 1.'}

In [ ]:
_res['items'][3]['metadata']['labels']
_res['items'][0]['status']['phase']

In [104]:
#30489
kubernetesTool.getPort(peername='invoice-chaincode-external', name = prefix + '-listen')

NameError: name 'prefix' is not defined

In [ ]:
kubernetespeer(name='invoice-net-chaincode-external').__dict__

In [108]:
#_res['items'][5]['status']['containerStatuses']
_res['items'][5]

{'apiVersion': 'v1',
 'kind': 'Pod',
 'metadata': {'creationTimestamp': '2021-10-22T09:19:35Z',
  'generateName': 'invoice-net-chaincode-external-576f8cbdd-',
  'labels': {'app': 'invoice-net-chaincode-external',
   'pod-template-hash': '576f8cbdd'},
  'managedFields': [{'apiVersion': 'v1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:generateName': {},
      'f:labels': {'.': {}, 'f:app': {}, 'f:pod-template-hash': {}},
      'f:ownerReferences': {'.': {},
       'k:{"uid":"6eb1c3d5-284f-4d41-9619-4ee45df70947"}': {'.': {},
        'f:apiVersion': {},
        'f:blockOwnerDeletion': {},
        'f:controller': {},
        'f:kind': {},
        'f:name': {},
        'f:uid': {}}}},
     'f:spec': {'f:containers': {'k:{"name":"invoice-net-chaincode-external"}': {'.': {},
        'f:env': {'.': {},
         'k:{"name":"CHAINCODE_ID"}': {'.': {}, 'f:name': {}, 'f:value': {}},
         'k:{"name":"CHAINCODE_SERVER_ADDRESS"}': {'.': {},
          'f:name': {},
          

In [78]:
for item in _res['items']:
    labels=item['metadata']['labels']
    if labels['app'] == 'invoice-net-chaincode-external':
        print(item)

{'apiVersion': 'v1', 'kind': 'Pod', 'metadata': {'creationTimestamp': '2021-10-22T09:19:35Z', 'generateName': 'invoice-net-chaincode-external-576f8cbdd-', 'labels': {'app': 'invoice-net-chaincode-external', 'pod-template-hash': '576f8cbdd'}, 'managedFields': [{'apiVersion': 'v1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:labels': {'.': {}, 'f:app': {}, 'f:pod-template-hash': {}}, 'f:ownerReferences': {'.': {}, 'k:{"uid":"6eb1c3d5-284f-4d41-9619-4ee45df70947"}': {'.': {}, 'f:apiVersion': {}, 'f:blockOwnerDeletion': {}, 'f:controller': {}, 'f:kind': {}, 'f:name': {}, 'f:uid': {}}}}, 'f:spec': {'f:containers': {'k:{"name":"invoice-net-chaincode-external"}': {'.': {}, 'f:env': {'.': {}, 'k:{"name":"CHAINCODE_ID"}': {'.': {}, 'f:name': {}, 'f:value': {}}, 'k:{"name":"CHAINCODE_SERVER_ADDRESS"}': {'.': {}, 'f:name': {}, 'f:value': {}}, 'k:{"name":"FABRIC_LOGGING_SPEC"}': {'.': {}, 'f:name': {}, 'f:value': {}}}, 'f:image': {}, 'f:imagePullPolicy': {}, 'f:n

## addPeer

In [35]:
def addPeer(peername):  
    if peername in peers.keys():
        return peers[peername]
    # Create configuration for new peer
    executeLocalCmd(['/bin/bash','-c','cd /home/jovyan/scray-ledger/containers/ && /home/jovyan/scray-ledger/containers/configure-deployment.sh -n ' + peername])
    # Start service
    executeKubectlCmd(['apply', '-f', '/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer-service.yaml'])
    # Create peer configuration
    res = createConfig(peername=peername,
                       peer_listen_port=getPort(peername=peername, name='peer-listen'),
                       peer_gossip_port=getPort(peername=peername, name='peer-gossip'))
    # Start new peer
    executeKubectlCmd(['apply', '-f','/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer.yaml'])
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer
    
# create, delete numbered peers
def createPeers(base_name,start=0, end=10):
    for x in range(start,end):
        peername = base_name + '-' + str(x)
        addPeer(peername)
         
def deletePeers(name):
    for x in range(0,10):
        peerneme = name + str(x)
        deletePeer(peerneme)    
  
def deleteAllPeers(config):
    for item in config['items']:
        try:
            if 'org_name' in item['data'].keys():
                deletePeer(item['data']['org_name'])
        except Exception as e:
            print(item['data'])
            

### config            
            
import subprocess
def deleteConfig(peername=''):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + peername])
        return str(output)
    except Exception as e:
        return str(e)  
        
    
def createConfig(peername='',peer_listen_port='',peer_gossip_port=''):
    try:  
        peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                          '--from-literal=hostname=' + peer_host_name,
                                          '--from-literal=org_name=' + peername,
                                          '--from-literal=data_share=hl-fabric-data-share-service:80',
                                          '--from-literal=CORE_PEER_ADDRESS=peer0.' + peer_host_name + ':' + peer_listen_port,
                                          '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=' + peer_host_name + ':' + peer_gossip_port,
                                          '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
        return str(output)
    except Exception as e:
        return str(e)      
    
  
##### execute

def executePeerCmd(pod_name,cmd, cli='scray-peer-cli'):
    try:  
        #cmd1 = "source /root/.bashrc && "
        cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd]
        #cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c'] + cmd
        print(toCmd(cmd1))
        output  = subprocess.check_output(cmd1)
        #output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec', '--stdin' ,'--tty', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd])
        return output.decode('ascii')
    except Exception as e:
        return str(e)

In [5]:
import kubernetesTool

class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name
        self.pod_name = kubernetesTool.getPod(name)
        self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        self.gossip_port = kubernetesTool.getPort(peername=name, name = prefix + '-gossip')
        # ????????
        #print('kubernetespeer addBashEnvToPeer')
        #addBashEnvToPeer(self)
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)      
        
class ordererpeer(kubernetespeer):
    def __init__(self, name='',prefix='peer'):
        super().__init__(name,prefix)
        self.host = 'orderer.example.com'
        self.orderer_intern = 7050
        
    # can change
    def get_ip(self, app='orderer-org1-scray-org'):
        return executeKubectlCmd(['get', 'pods','-l', 'app=' + app,'-o','jsonpath=\'{.items[*].status.podIP}\''])   
    

In [6]:
def toCmd(strlist):
    return ''.join((str(e) + ' ') for e in strlist)[:-1]

#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME         
def joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip='10.15.136.41'):          
    cmd = toCmd(['/mnt/conf/peer_join.sh', orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip])
    print(cmd)
    return executePeerCmd(peer.pod_name, cmd)

def endorse_peer(peer=None, orderer_ip='',channel_name='',shared_fs_host='',ext_peer_ip='10.15.136.41'):
    print(peer.name)
    peername = peer.name
    peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
    cmd = toCmd(['/mnt/conf/orderer/scripts/inform_existing_nodes.sh',orderer_ip,channel_name,peer.name,shared_fs_host,ext_peer_ip,peer_host_name]) 
    #print(cmd)
    return executePeerCmd(orderer.pod_name,cmd,cli='scray-orderer-cli')

def add_channel(name):
    cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050']
    cmd = toCmd(cmd1)
    return executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

## create peers

In [26]:
orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer')


orderer_ip=orderer.get_ip()
orderer_hostname=orderer.host
#orderer_port=orderer.orderer_intern
orderer_port= orderer.listen_port

channel_name='super55'
shared_fs_host='10.14.128.38:30080'

In [10]:
def endorse_and_join(peer):    
    res1 = endorse_peer(peer=peer,orderer_ip=orderer_ip,channel_name=channel_name,shared_fs_host='10.14.128.38:30080')   
    res2  = joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host)
    return res1,res2

In [12]:
peers = {}

In [14]:
createPeers('aabby',start=0, end=2)

In [27]:
## endorse and join

import ipywidgets as widgets
out2 = widgets.Output()

with out2:
    for key, value in peers.items():
        endorse_and_join(value)

In [28]:
out2

Output(outputs=({'name': 'stdout', 'text': "Aabby-0\n/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-sc…

In [16]:
peername='zalya-3'
peer = kubernetespeer(name=peername)
peers[peername] = peer

In [ ]:
'zalya-0'

In [ ]:
peers['zalya-1'].__dict__
#peers

In [ ]:
addPeer('zalya-3')

In [ ]:
orderer.__dict__

In [ ]:
kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/inform_existing_nodes.sh $ORDERER_IP $CHANNEL_NAME $PEER_NAME $SHARED_FS_HOST $EXT_PEER_IP $PEER_HOST_NAME

In [ ]:
EER_POD_NAME=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
 ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
 ORDERER_PORT=7050
 PEER_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
 kubectl exec --stdin --tty $PEER_POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $ORDERER_PORT $CHANNEL_NAME $SHARED_FS_HOST $EXT_PEER_IP

In [ ]:
endorse_and_join(peers['zalya-3'])

In [ ]:
installAndAproveChaincode(peers['zalya-1'],orderer,channel_name)


In [ ]:
#out2.outputs[1]['text'].splitlines()

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-5f97c57d44-j5mbs -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super40 kalya-0 10.14.128.38:30080 10.15.136.41 kalya-0.kubernetes.research.dev.seeburger.de

In [ ]:
name='peer-gossip'
callopt= 'jsonpath=\"{.spec.ports[?(@.name==\'' + name + '\')].nodePort}\"'
callopt

## install chaincode

In [133]:
def installAndAproveChaincode(peer,orderer,channel_name,cc_host='10.14.128.38',pkgid='',cc_hostname='asset-transfer-basic.org1.example.com',cc_label='basic_1.0',shared_fs='kubernetes.research.dev.seeburger.de:30080', sequence=1):
    #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK
    #strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.orderer_intern), channel_name ]
    strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.listen_port), channel_name,pkgid,cc_hostname, cc_label,shared_fs, str(sequence) ]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

def commitChaincode(peer,channel_name,sequence=1,pkgid=''):
    strlist = ['/mnt/conf/peer/cc_commit.sh', channel_name, str(sequence),pkgid ]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

In [157]:
for key, peer in peers.items():
    executePeerCmd(peer.pod_name,'apk add curl')
    strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh', '-o', '/mnt/conf/install_and_approve_cc.sh']
    cmd = toCmd(strlist)
    print(cmd)
    output = executePeerCmd(peer.pod_name,  cmd)
    executePeerCmd(peer.pod_name,'chmod 755 /mnt/conf/install_and_approve_cc.sh')
    strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh', '-o', '/mnt/conf/peer/cc_commit.sh']
    cmd = toCmd(strlist)
    print(cmd)
    output = executePeerCmd(peer.pod_name,  cmd)
    executePeerCmd(peer.pod_name,'chmod 755 /mnt/conf/peer/cc_commit.sh')

/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c apk add curl
curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh -o /mnt/conf/install_and_approve_cc.sh
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh -o /mnt/conf/install_and_approve_cc.sh


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1515  100  1515    0     0   6446      0 --:--:-- --:--:-- --:--:--  6446


/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c chmod 755 /mnt/conf/install_and_approve_cc.sh
curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh -o /mnt/conf/peer/cc_commit.sh
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh -o /mnt/conf/peer/cc_commit.sh


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   812  100   812    0     0   3530      0 --:--:-- --:--:-- --:--:--  3515


/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c chmod 755 /mnt/conf/peer/cc_commit.sh
/home/jovyan/work/usr/bin/kubectl exec -t aabby-1-5ff5b8677f-kvfl7 -c scray-peer-cli -- /bin/sh -c apk add curl
curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh -o /mnt/conf/install_and_approve_cc.sh
/home/jovyan/work/usr/bin/kubectl exec -t aabby-1-5ff5b8677f-kvfl7 -c scray-peer-cli -- /bin/sh -c curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh -o /mnt/conf/install_and_approve_cc.sh


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1515  100  1515    0     0  40945      0 --:--:-- --:--:-- --:--:-- 40945


/home/jovyan/work/usr/bin/kubectl exec -t aabby-1-5ff5b8677f-kvfl7 -c scray-peer-cli -- /bin/sh -c chmod 755 /mnt/conf/install_and_approve_cc.sh
curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh -o /mnt/conf/peer/cc_commit.sh
/home/jovyan/work/usr/bin/kubectl exec -t aabby-1-5ff5b8677f-kvfl7 -c scray-peer-cli -- /bin/sh -c curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh -o /mnt/conf/peer/cc_commit.sh


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   812  100   812    0     0  21368      0 --:--:-- --:--:-- --:--:-- 21945


/home/jovyan/work/usr/bin/kubectl exec -t aabby-1-5ff5b8677f-kvfl7 -c scray-peer-cli -- /bin/sh -c chmod 755 /mnt/conf/peer/cc_commit.sh


In [140]:
shared_fs='kubernetes.research.dev.seeburger.de:30080'
cc_hostname='invoice-net.org1.example.com'
cc_host='10.14.128.38'
cc_service_name='hl-fabric-cc-external-invoice-net'
cc_port = executeKubectlCmd(['get', 'service', cc_service_name, '-o', 'jsonpath="{.spec.ports[?(@.name==\'chaincode\')].nodePort}"'], decode='json')
cc_label='basic_1.0'

In [139]:
cc_port

'30489'

In [141]:
strlist = ['curl','-s','--user','scray:scray','http://' + shared_fs + '/cc_descriptions/' + cc_hostname + '_' + cc_label + '/description-hash.json']
cmd = toCmd(strlist)
output = executePeerCmd(peer.pod_name,  cmd)
pkgid = json.loads(output[:-1])['description-hash']

/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c curl -s --user scray:scray http://kubernetes.research.dev.seeburger.de:30080/cc_descriptions/invoice-net.org1.example.com_basic_1.0/description-hash.json


In [158]:
for key, peer in peers.items():
    installAndAproveChaincode(peer,orderer,channel_name,cc_host=cc_host,pkgid=pkgid,cc_hostname=cc_hostname,cc_label=cc_label,shared_fs=shared_fs, sequence=1)

/mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   431  100   431    0     0  15962      0 --:--:-- --:--:-- --:--:-- 15962
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
Error: proposal failed with status: 500 - failed to invoke backing implementation of 'ApproveChaincodeDefinitionForMyOrg': attempted to redefine uncommitted sequence (7) for namespace basic with unchanged content


/mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1
/home/jovyan/work/usr/bin/kubectl exec -t aabby-1-5ff5b8677f-kvfl7 -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   431  100   431    0     0  39181      0 --:--:-- --:--:-- --:--:-- 39181
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
Error: proposal failed with status: 500 - failed to invoke backing implementation of 'ApproveChaincodeDefinitionForMyOrg': attempted to redefine uncommitted sequence (7) for namespace basic with unchanged content


In [159]:

peer = peers['aabby-0']
#peer = list(peers.values())[0]
commitChaincode(peer,channel_name,sequence=1,pkgid=pkgid)

/mnt/conf/peer/cc_commit.sh super55 1 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc_commit.sh super55 1 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77


2021-11-15 14:24:11.719 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [e7101614e66ecb16e9fc446c60ba593a1997884c797ed4982b70f4a30f43e890] committed with status (VALID) at peer0.aabby-0.kubernetes.research.dev.seeburger.de:30149


"Installed chaincode\nInstalled chaincodes on peer:\nPackage ID: basic_1.0:19e4e39a2a3e13f9cd130d9d8b57207a2ac09b533f9fcc87506a253719a7b739, Label: basic_1.0\nPackage ID: basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77, Label: basic_1.0\nPackage ID: basic_1.0:fd7a1dd538bca88611519d55085d7dcc59218bfcdfc32d1d1adc7f9359e69240, Label: basic_1.0\nChaincode definition for chaincode 'basic', version '1.0', sequence '7' on channel 'super55' approval status by org:\nAdminOrgMSP: false\naabby-0MSP: true\naabby-1MSP: true\n"

In [ ]:
#channel_name
#orderer.__dict__
#peers['ralya-0'].pod_name
peers['zalya-0'].__dict__ , channel_name

In [ ]:
installAndAproveChaincode(peers['zalya-1'],orderer,channel_name,sequence=2)


In [ ]:
#commitChaincode(peers['zalya-1'],channel_name,sequence=2)
chaincode_init(peers['zalya-1'],channel_name)

In [ ]:
peers['zalya-2'].pod_name

In [ ]:
def downloadToPeer(peer, orgs, execute_org='schalke3'):
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name=getCrtName(org))))   

In [45]:
peers

{'aabby-0': <__main__.kubernetespeer at 0x7fe41472ccd0>,
 'aabby-1': <__main__.kubernetespeer at 0x7fe41463bbb0>}

In [18]:
peer = peers['zalya-0']
executePeerCmd(peer.pod_name,'apk add curl')

/home/jovyan/work/usr/bin/kubectl exec -t zalya-0-6cf64f96b4-vv4mp -c scray-peer-cli -- /bin/sh -c apk add curl


'fetch http://dl-cdn.alpinelinux.org/alpine/v3.12/main/x86_64/APKINDEX.tar.gz\nfetch http://dl-cdn.alpinelinux.org/alpine/v3.12/community/x86_64/APKINDEX.tar.gz\n(1/1) Installing curl (7.79.1-r0)\nExecuting busybox-1.31.1-r16.trigger\nOK: 29 MiB in 28 packages\n'

In [20]:
peer.pod_name

'zalya-0-6cf64f96b4-vv4mp'

In [ ]:
peer=kubernetespeer(name='oalya-0')
channel_name='super45'

In [ ]:
peer.name, channel_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/cc-basic-interaction.sh  super43

In [70]:
#Call init method in chaincode
def chaincode_init(peer,channel_name):
    strlist = ['/mnt/conf/peer/cc-basic-interaction.sh', channel_name ]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

chaincode_init(peer,channel_name)

/mnt/conf/peer/cc-basic-interaction.sh super55
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc-basic-interaction.sh super55


Error: endorsement failure during invoke. response: status:500 message:"make sure the chaincode basic has been successfully defined on channel super55 and try again: chaincode definition for 'basic' exists, but chaincode is not installed" 
Error: endorsement failure during query. response: status:500 message:"make sure the chaincode basic has been successfully defined on channel super55 and try again: chaincode definition for 'basic' exists, but chaincode is not installed" 
command terminated with exit code 1


"Command '['/home/jovyan/work/usr/bin/kubectl', 'exec', '-t', 'aabby-0-6859c54c76-gvdqw', '-c', 'scray-peer-cli', '--', '/bin/sh', '-c', '/mnt/conf/peer/cc-basic-interaction.sh super55']' returned non-zero exit status 1."

In [ ]:
externalbuilder/builds/basic_1.0-21ee1256844f847e196af6aec17c5cafa76d8bd652d6ef42c3058e52571030b4/release/chaincode/server/c

In [ ]:
peers['nalya-0'].pod_name

In [ ]:
CHANNEL_NAME='super44'
#ORDERER_NAME=orderer.example.com
IP_CC_SERVICE='10.14.128.38'         # Host where the chaincode is running
#PEER_POD=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
#ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')
#ORDERER_LISTEN_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
#ORDERER_HOST=orderer.example.com
EXT_PEER_IP="10.14.128.38"
PEER_POD="nalya-0-5494459f47-v7msj"

ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
ORDERER_PORT=30081
ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')

kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $ORDERER_IP $ORDERER_HOST $ORDERER_PORT $CHANNEL_NAME 

In [ ]:
#peers.keys()
#[elem for elem in peers.values()]
list(peers.values())[0].__dict__

# invoice functions

In [49]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh  $CHANNEL_NAME $INVOICE_ID

def get_certificate_string(peername):
    return 'x509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
    #return 'x509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE,C=DEdx509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

def chaincode_add_invoice(peer,channel_name,invoice_id,buyer):
    strlist = ['/mnt/conf/peer/add-invoice.sh', channel_name,invoice_id,get_certificate_string(buyer)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 


## invoice usecase

In [170]:
peernames = list(peers.keys())
peernames

['aabby-0', 'aabby-1']

In [162]:
import json

def getHostname(peer):
    return executePeerCmd(peer.pod_name,'echo $HOSTNAME')[:-1]

def getExport_MSPCONFIGPATH(hostname,user='User1'):
    #user = 'User1'
    #user = 'otto'
    #path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/User1@' + hostname + '/msp/'
    path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + user + '@' + hostname + '/msp/'
    return 'export CORE_PEER_MSPCONFIGPATH=' + path
    
def getArgList(asset):
    if isinstance(asset, dict):
        asset = asset.values()
    return '[' + ''.join(('\"' + str(e) + '\",') for e in asset)[:-1] + ']' 

def invoke(peer,callopt):
    export_cmd = getExport_MSPCONFIGPATH(getHostname(peer))
    cmd0 = str('\'' + callopt + "\'")
    strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
    cmd = toCmd(strlist)
    print(cmd)  
    output = executePeerCmd(peer.pod_name, export_cmd + '  && ' + cmd)
    return output

def query(peer, callopt, user='User1'):
    try:
        export_cmd = getExport_MSPCONFIGPATH(getHostname(peer),user=user)
        #hostname = getHostname(peer)
        #path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + 'User1' + '@' + hostname + '/msp/'
        cmd0 = str('\'' + callopt + "\'")
        strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0]
        #strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0,'--cafile',path + 'cacerts/ca.zalya-0.kubernetes.research.dev.seeburger.de-cert.pem' , '--certfile',path + 'signcerts/User1@zalya-0.kubernetes.research.dev.seeburger.de-cert.pem' ,'--keyfile', path + 'keystore/priv_sk']
        cmd = toCmd(strlist)
        print(cmd)
        output = executePeerCmd(peer.pod_name, export_cmd + '  && ' + cmd)
        #output = executePeerCmd(peer.pod_name, cmd)           
        return json.loads(output)
    except Exception as e:
        #print(e)
        return output
    
def getAllAssets(peer):
    callopt= '{"function":"GetAllAssets","Args":[' + ']}'
    return query(peer,callopt)    
    
def readAsset(peer, asset='asset1'):
    callopt= '{"function":"ReadAsset","Args":[' + '\"' + asset + '\"' + ']}'
    return query(peer,callopt)

def readAssetTest(peer, asset='asset1'):
    callopt= '{"function":"ReadAssetTest","Args":[' + '\"' + asset + '\"' + ']}'
    return query(peer,callopt)
    
def createAsset(peer,arglist=''):
    callopt= '{"function":"CreateAsset","Args":' + arglist + '}'
    return invoke(peer,callopt)

def updateAsset(peer,arglist=None):
    callopt= '{"function":"UpdateAsset","Args":' + getArgList(asset) + '}'
    return invoke(peer,callopt)

def appendRole(peer,name='',role='Buyer'):
    print(getArgList([role,name]))
    #callopt= '{"function":"AppendRole","Args":' + getArgList([role,name]) + '}'
    callopt= '{"function":"AppendRole","Args":' + getArgList([name,role]) + '}'
    return invoke(peer,callopt)

def getRoles(peer,name=''):
    #callopt= '{"function":"GetRoles","Args":[' + ']}'
    callopt= '{"function":"GetRoles","Args":' + getArgList([name]) + '}'
    return query(peer,callopt)

def getAllRoles(peer):
    callopt= '{"function":"GetAllRoles","Args":[' + ']}'
    return query(peer,callopt)


def getSubmittingClientIdentity(peer):
    callopt= '{"function":"GetSubmittingClientIdentity","Args":[' + ']}'
    return query(peer,callopt)

In [233]:
#peers
peer = peers['aabby-0']
#getHostname(peer)

In [216]:
callopt= '{"function":"GetRoles","Args":' + getArgList([]) + '}'
query(peer,callopt)

/home/jovyan/work/usr/bin/kubectl exec -t aabby-1-5ff5b8677f-kvfl7 -c scray-peer-cli -- /bin/sh -c echo $HOSTNAME
peer chaincode query -C super55 -n basic -c '{"function":"GetRoles","Args":[]}'
/home/jovyan/work/usr/bin/kubectl exec -t aabby-1-5ff5b8677f-kvfl7 -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/aabby-1.kubernetes.research.dev.seeburger.de/users/User1@aabby-1.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super55 -n basic -c '{"function":"GetRoles","Args":[]}'


Error: endorsement failure during query. response: status:500 message:"Error handling success response. Value did not match schema:\n1. return.roles: Invalid type. Expected: array, given: null" 
command terminated with exit code 1


'Command \'[\'/home/jovyan/work/usr/bin/kubectl\', \'exec\', \'-t\', \'aabby-1-5ff5b8677f-kvfl7\', \'-c\', \'scray-peer-cli\', \'--\', \'/bin/sh\', \'-c\', \'export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/aabby-1.kubernetes.research.dev.seeburger.de/users/User1@aabby-1.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super55 -n basic -c \\\'{"function":"GetRoles","Args":[]}\\\'\']\' returned non-zero exit status 1.'

In [235]:
#owner = readAssetTest(peer,'asset6') ['owner']
name = getSubmittingClientIdentity(peer)[:-1]
#appendRole(peer,name=name,role='Seller')
appendRole(peer,name=name,role='Buyer')

/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c echo $HOSTNAME
peer chaincode query -C super55 -n basic -c '{"function":"GetSubmittingClientIdentity","Args":[]}'
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/aabby-0.kubernetes.research.dev.seeburger.de/users/User1@aabby-0.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super55 -n basic -c '{"function":"GetSubmittingClientIdentity","Args":[]}'
["Buyer","x509::CN=User1@aabby-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.aabby-0.kubernetes.research.dev.seeburger.de,O=aabby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE"]
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c echo $HOSTNAME
peer chaincode invoke -o orderer.example.com:30081 --tls --caf

2021-11-17 10:18:31.204 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


''

In [240]:
getAllRoles(peer)
#getRoles(peer,name=name)
#getSubmittingClientIdentity(peer)
#getAllAssets(peer)

/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c echo $HOSTNAME
peer chaincode query -C super55 -n basic -c '{"function":"GetAllRoles","Args":[]}'
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/aabby-0.kubernetes.research.dev.seeburger.de/users/User1@aabby-0.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super55 -n basic -c '{"function":"GetAllRoles","Args":[]}'


[{'name': 'x509::CN=User1@aabby-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.aabby-0.kubernetes.research.dev.seeburger.de,O=aabby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
  'roles': ['Buyer']},
 {'name': 'x509::CN=User1@aabby-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.aabby-1.kubernetes.research.dev.seeburger.de,O=aabby-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
  'roles': ['Seller', 'Factor', 'Buyer']}]

In [172]:
peer  = peers[peernames[0]]        
asset = readAssetTest(peer)    
asset['ID']='asset6'
asset['buyer']='x509::CN=User1@zalya-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.zalya-1.kubernetes.research.dev.seeburger.de,O=zalya-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
createAsset(peer,getArgList(asset))

/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c echo $HOSTNAME
peer chaincode query -C super55 -n basic -c '{"function":"ReadAssetTest","Args":["asset1"]}'
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/aabby-0.kubernetes.research.dev.seeburger.de/users/User1@aabby-0.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super55 -n basic -c '{"function":"ReadAssetTest","Args":["asset1"]}'
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c echo $HOSTNAME
peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pem -C super55 -n basic -c '{"function":"CreateAsset","Args":["asset6","company","x509::CN=User1@zalya-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.zalya-1.kubernetes.researc

2021-11-15 15:01:47.597 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


''

In [ ]:
peers

In [239]:
getAllAssets(peers[peernames[0]])

/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c echo $HOSTNAME
peer chaincode query -C super55 -n basic -c '{"function":"GetAllAssets","Args":[]}'
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/aabby-0.kubernetes.research.dev.seeburger.de/users/User1@aabby-0.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super55 -n basic -c '{"function":"GetAllAssets","Args":[]}'


[{'Key': 'asset1',
  'Record': {'ID': 'asset1',
   'owner': 'company',
   'buyer': '',
   'hash': 0,
   'invoiceNumber': '0',
   'tax': 0,
   'netto': 0,
   'countryOrigin': 'DE',
   'countryBuyer': '',
   'status': '',
   'received': False,
   'receivedOrder': False,
   'sold': False,
   'claimPaid': False,
   'claimPaidBy': '',
   'taxExemptionReason': '',
   'taxReceived': False}},
 {'Key': 'asset6',
  'Record': {'ID': 'asset6',
   'owner': 'x509::CN=User1@aabby-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.aabby-0.kubernetes.research.dev.seeburger.de,O=aabby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
   'buyer': 'x509::CN=User1@zalya-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.zalya-1.kubernetes.research.dev.seeburger.de,O=zalya-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
   'hash': 0,
   'invoiceNumber': '0',
   'tax': 0,
   'netto': 0,
   'countryOrigin': 'DE',
   'cou

In [ ]:
asset=readAsset(peer, asset='asset1')
asset['sold'] = True
updateAsset(peer, asset)

In [176]:
asset = readAsset(peers[peernames[0]], asset='asset6')

from facturxWidget import facturx_widgets 

import json
with open('asset.json', 'w', encoding='utf-8') as f:
    json.dump(asset, f, ensure_ascii=False, indent=4)

/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c echo $HOSTNAME
peer chaincode query -C super55 -n basic -c '{"function":"ReadAsset","Args":["asset6"]}'
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/aabby-0.kubernetes.research.dev.seeburger.de/users/User1@aabby-0.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super55 -n basic -c '{"function":"ReadAsset","Args":["asset6"]}'


In [ ]:
import json

with open('asset.json') as json_file:
    data = json.load(json_file)

data    

In [ ]:
readAsset(peer) 

In [ ]:
callopt= '{"function":"ReadAsset","Args":[' + '\"' + 'asset1' + '\"' + ']}'
cmd0 = str('\'' + callopt + "\'")
strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
cmd = toCmd(strlist)

executePeerCmd(peers[pernames[0]].pod_name, export_cmd + '  && ' + cmd)

In [ ]:
 CreateAsset(ctx contractapi.TransactionContextInterface, id, owner string,  hash int,
             invoiceNumber string, vat float32, netto float32, countryOrigin string, countryReceiver string, received bool,
        receivedOrder bool, sold bool, claimPaid bool, claimPaidBy string, taxExemptionReason string,taxReceived bool

In [ ]:
def getPeer0Address(peer):
    addr='peer0.kubernetes.research.dev.seeburger.de'
    return str(addr + ':' + peer.listen_port)

def readAsset(peer, asset='asset1'):
    #callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    callopt= '{"function":"ReadAsset","Args":[' + '\"' + asset + '\"' + ']}'
    cmd0 = str('\'' + callopt + "\'")
    try:
        #cmd1= 'CORE_PEER_ADDRESS=\"peer0.kubernetes.research.dev.seeburger.de:31866\" && ' 
        #cmd1 = 'CORE_PEER_ADDRESS=\"' + getPeer0Address(peer) + '\" && '
        cmd1 = 'CORE_PEER_MSPCONFIGPATH=' + path + '\" && '
        #cmd2 = ['peer', 'chaincode','query', '-C', channel_name, '-n', 'basic', '-c']
        cmd2 = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem', '-C',channel_name, '-n', 'basic', '-c']
   
        cmd = toCmd(cmd2) + ' ' + cmd0
        print(cmd)
        #output = executePeerCmd(peer.pod_name, cmd1 + cmd)
        #print(output)
        if output[0] == '{':
            return json.loads(output)
        return cmd
    except Exception as e:
        print(e)
        
asset = readAsset(peers[pernames[0]], 'asset1')
executePeerCmd(peers[pernames[0]].pod_name,asset)

In [ ]:
invoice_id='046'
buyer=pernames[0]

strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem', '-C',channel_name,invoice_id,get_certificate_string(buyer)]
print(toCmd(strlist))
executePeerCmd(peers[pernames[0]].pod_name, toCmd(strlist))

In [ ]:
CHANNEL_ID=$1
ASSET_ID=$2
PRODUCT_BUYER=$3

export PKGID=basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee

export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/$HOSTNAME/users/User1@$HOSTNAME/msp/

peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pem -C $CHANNEL_ID -n basic -c '{"function":"CreateAsset","Args":["'${ASSET_ID}'", "'${PRODUCT_BUYER}'"]}'

## invoice usecase (Simple)

In [ ]:

pernames = list(peers.keys())
chaincode_add_invoice(peers[pernames[0]],channel_name,'046',pernames[1])

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -t galya-0-6cc848b49-pv2dn -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh super13 006 x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US

In [ ]:
get_certificate_string(pernames[1])

#'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

#, 'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

In [ ]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  $CHANNEL_NAME $INVOICE_ID
def chaincode_read_invoice(peer,channel_name):
    strlist = ['/mnt/conf/peer/get-my-invoices.sh', channel_name]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 
chaincode_read_invoice(peers[pernames[0]],channel_name)

In [ ]:
def chaincode_transfer_invoice(peer,channel_name,invoice_id,new_owner):
    strlist = ['/mnt/conf/peer/transfer_invoice.sh', channel_name,invoice_id,get_certificate_string(new_owner)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

pernames = list(peers.keys())
chaincode_transfer_invoice(peers[pernames[0]],channel_name,'045',pernames[2])

In [ ]:
get_certificate_string(pernames[2])
NEW_OWNER='x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/transfer_invoice.sh  super43 045 $NEW_OWNER

In [ ]:
kubectl exec --stdin --tty 'lalya-0-7c887cffb8-fpw7h' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-1-7c76dff8c-pzx78' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-2-5f8b45dbc4-52tt7' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045


In [ ]:
peers[pernames[2]].pod_name

## add channel

In [23]:
orderer.pod_name

'orderer-org1-scray-org-5cb87587c4-q95mm'

In [25]:
!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-5cb87587c4-q95mm -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super55 orderer.example.com 30081

2021-11-11 08:32:57.480 UTC [common.tools.configtxgen] main -> INFO 001 Loading configuration
2021-11-11 08:32:57.500 UTC [common.tools.configtxgen.localconfig] Load -> INFO 002 Loaded configuration: /mnt/conf/orderer/configtx.yaml
2021-11-11 08:32:57.500 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 003 Generating new channel configtx
2021-11-11 08:32:57.503 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 004 Writing new channel tx
2021-11-11 08:32:57.568 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-11-11 08:32:57.618 UTC [cli.common] readBlock -> INFO 002 Expect block, but got status: &{NOT_FOUND}
2021-11-11 08:32:57.621 UTC [channelCmd] InitCmdFactory -> INFO 003 Endorser and orderer connections initialized
2021-11-11 08:32:57.823 UTC [cli.common] readBlock -> INFO 004 Expect block, but got status: &{SERVICE_UNAVAILABLE}
2021-11-11 08:32:57.829 UTC [channelCmd] InitCmdFactory -> INFO 005 Endorser and or

In [ ]:
add_channel('super20')

In [ ]:
name = 'super30'
cmd = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050'])
cmd, orderer.pod_name

In [ ]:
executePeerCmd(orderer.pod_name, cmd, cli='scray-orderer-cli')

In [ ]:
executePeerCmd(orderer.pod_name, 'peer channel list', cli='scray-orderer-cli')

In [ ]:
cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh super20 orderer.example.com 7050']
executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

In [ ]:
#cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c','/mnt/conf/orderer/scripts/create_channel.sh super7 orderer.example.com 7050']

#cmd2 = toCmd(['ls','-a'])
cmd2 = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', 'super7', 'orderer.example.com', '7050'])

cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c',cmd2]


print(toCmd(cmd1))
#subprocess.check_output(cmd1)
subprocess.run(cmd1, stdout = subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -it orderer-org1-scray-org-5f97c57d44-l6d5l -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super6 orderer.example.com 7050


In [ ]:
orderer.pod_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty 'orderer-org1-scray-org-5f97c57d44-j5mbs' -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh 'super' orderer.example.com '7050'

In [ ]:
# Create Peer

peername='dalya8'
addPeer(peername)
peers[peername].__dict__

In [ ]:
executeKubectlCmd(['get', 'pods','-l', 'app=orderer-org1-scray-org','-o','jsonpath=\'{.items[*].status.podIP}\''])  

In [ ]:
peers[peername].__dict__

In [ ]:
orderer.__dict__

## Export channel configuration

In [ ]:
#https://github.com/scray/scray/blob/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/README.md#export-data

SHARED_FS_HOST='10.14.128.38:30080'
_result = SHARED_FS_HOST + '/channel/configuration/$CHANNEL_NAME/config.json'

#kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/publish_channel_conf.sh $CHANNEL_NAME $SHARED_FS_HOST

def export_channel_configuration(peer,channel_name,SHARED_FS_HOST):
    strlist = ['/mnt/conf/orderer/scripts/publish_channel_conf.sh', channel_name,SHARED_FS_HOST]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

#_peer = peers[list(peers.keys())[0]]
_peer = orderer
channel_name = 'super'
export_channel_configuration(_peer,channel_name,SHARED_FS_HOST)

In [ ]:
#list(peers)[0]


# Enroll users

In [33]:
import subprocess
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
   

def uploadFromLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                          '-T', local_path + '/' + local_name, 
                                          'http://' + host + '/' + remote_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)   
                    
#downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

def downloadToLocalCmdList(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', remote_name='' ,remote_path='/'):  
    return ['curl', '--user',user + ':' + pwd, 'http://' + host + '/' + remote_path + '/' + remote_name, '-o', local_path + '/' + remote_name]


In [ ]:
# download walletcreator to peers
def uploadWalletToolToSharepoint():
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator/target', local_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar' ,remote_path='/')
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator', local_name='cert-creator.sh' ,remote_path='upload')

def installPeerWalletTool(peer):
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar'))) 
    #executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='cert-creator.sh'))) 
    executePeerCmd(peer.pod_name,'apk add openssl')
    executePeerCmd(peer.pod_name,'apk add openjdk8-jre')
    # chmod 755
    
def createPeerUser(peer,user, admin='admin',
    cert='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/ca.kubernetes.research.dev.seeburger.de-cert.pem',
    key='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/priv_sk'):
        cmd = ['/tmp/cert-creator.sh','--cacert',cert,'--cakey',key,'--new-user-crt',user,'--organizational-unit',admin,
               '--create-wallet','true','--wallet-creator-lib-path', '/tmp']
        return executePeerCmd(peer.pod_name,toCmd(cmd))
    
installPeerWalletTool(peers['zalya-0'])    
#uploadWalletToolToSharepoint()  
#createPeerUser(peers['oly0'],'bretten')     

In [ ]:
peer = peers['oly0']
# build local , upload tool to sharepoint 
#uploadWalletToolToSharepoint()
#installPeerWalletTool(peer)

createPeerUser(peer,'bretten')   